https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [ ]:
# copy token here
token = "b26aa5422a903025651eed18bab686470c0e931156befe595e9d6c3728ae9bd67a9a4d4bfc05376982930"

In [6]:
poll_group_id = "-172053584"

# VK API

In [70]:
import vk
from vk.exceptions import VkAPIError

In [19]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting the whole wall

### Creating data dir if it doesn't exist

In [59]:
import os

In [62]:
os.makedirs("data", exist_ok=True)

### Requesting first 100

In [18]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

In [20]:
group_wall_json = api.wall.get(**kwargs)

In [21]:
len(group_wall_json["items"])

100

In [22]:
# group_wall_json

{'count': 503,
 'items': [{'attachments': [{'poll': {'anonymous': 0,
      'answer_id': 0,
      'answers': [{'id': 1086523657, 'rate': 71.29, 'text': 'Да', 'votes': 72},
       {'id': 1086523658, 'rate': 28.71, 'text': 'Нет', 'votes': 29}],
      'created': 1550061631,
      'id': 323103259,
      'owner_id': -172053584,
      'question': 'Нормально ли дарить Валентинки друзьям противоположного пола?',
      'votes': 101},
     'type': 'poll'}],
   'comments': {'count': 4},
   'date': 1550081105,
   'from_id': -172053584,
   'id': 1659,
   'likes': {'count': 0},
   'marked_as_ads': 0,
   'owner_id': -172053584,
   'post_type': 'post',
   'reposts': {'count': 0},
   'text': '#предложка_кпсо',
   'views': {'count': 214}},
  {'attachments': [{'photo': {'access_key': '3952f2f2d35a3ed8c5',
      'album_id': -7,
      'date': 1550051856,
      'height': 431,
      'id': 456239530,
      'owner_id': -172053584,
      'photo_130': 'https://pp.userapi.com/c847124/v847124403/19df2f/2S5Lro_O-Tc.

In [23]:
import time

while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    time.sleep(3)

Saving the result:

In [26]:
import json

In [29]:
with open("data/group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [63]:
with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [64]:
# wall_post = group_wall_json["items"][1]
# poll = get_poll(wall_post)
# poll_voters = api.polls.getVoters(**kwargs)
# poll_voters

In [65]:
def get_poll(wall_post):
    if "attachments" not in wall_post:
        return None
    for attachment in wall_post["attachments"]:
        if attachment["type"] != "poll":
            continue
        poll = attachment["poll"]
        return poll
    return None

In [73]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
#         print(vk_error)
        time.sleep(1)
        continue
    with open("data/poll_{}.json".format(poll["id"]), "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)

250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '322587640', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1084808611,1084839731', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
Processed 20/2 polls
Processed 40/2 polls
Processed 60/2 polls
Processed 80/2 polls
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '321483236', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1081150430', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '321430282', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1080975140,1080975141,1080975142,1080975143,1080975144,1080975145,1080975146', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. requ

250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '309374584', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1040830483,1040830484', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '309239581', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1040379234,1040379235,1040379236,1040379237,1040379238,1040380290', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '309238335', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1040375027,1040375028,1040375029,1040375030,1040375031,1040375032,1040375033,1040375034,1040375035', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_par

250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '308572150', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1038144348,1038144349', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '308578791', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1038166927,1038166928,1038166929,1038166930,1038166931,1038166932,1038166933,1038166934', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '308574790', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1038153424,1038153425,1038153426,1038153427,1038153428,1038153429,1038153430,1038153431,1038153432', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to po

250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '307788523', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1035519579,1035519580,1035519668', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '307736245', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1035344414,1035344415,1035344416,1035344417,1035347790', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '307727698', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1035315622,1035315623,1035315624', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'po

250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '306985419', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1032855871,1032855872,1032855873,1032855874,1032855875', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '306960919', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1032775633,1032775634,1032775635,1032775636', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to poll denied. request_params = {'oauth': '1', 'method': 'polls.getVoters', 'v': '5.74', 'poll_id': '306936260', 'owner_id': '-172053584', 'count': '1000', 'answer_ids': '1032703091,1032703092,1032703093,1032703094,1032703095,1032703096,1032703097,1032703098,1032703099,1032703100', 'fields': 'nickname, screen_name, sex, photo, photo_medium, photo_big'}
250. Access to po

# Processing and squeezing data

### Squeezing all users:

In [100]:
all_ids_to_users = {}
all_ids = []
for filename in os.listdir("data"):
    if "poll_" not in filename:
        continue
    poll_id = int(filename.split("poll_")[1].split(".")[0])
    with open("data/" + filename, "r") as f:
        poll = json.load(f)
    
    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        answer_id_to_user = list(map(lambda vote: (vote["id"], vote), answer["users"]["items"]))
        all_ids_to_users.update(dict(answer_id_to_user))
        all_ids.extend(map(lambda item: item[0], answer_id_to_user))

In [101]:
from collections import Counter
cnt = Counter(all_ids)

In [102]:
for user_id in cnt:
    all_ids_to_users[user_id]["num_votes"] = cnt[user_id]

In [103]:
with open("data/users.json", "w") as f:
    json.dump(all_ids_to_user, f)

### Building "sparse" relativity matrix.

In [105]:
polls = []
for filename in os.listdir("data"):
    if "poll_" not in filename:
        continue
    poll_id = int(filename.split("poll_")[1].split(".")[0])
    with open("data/" + filename, "r") as f:
        poll = json.load(f)
    
    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        answers_to_ids.append(list(map(lambda vote: vote["id"], answer["users"]["items"])))
    polls.append(answers_to_ids)

Now we have variable 'polls' which is [POLL_ID \* ANSWER_ID \* USER] where ids are not real but just indexation.

In [108]:
with open("data/vote_matrix.json", "w") as f:
    json.dump(polls, f)